In [6]:
import numpy as np
import math
import matplotlib.pyplot as plt
import imageio
import os
import time
import sys

from liblibra_core import *
import libra_py.models.Holstein as Holstein
import util.libutil as comn
from libra_py import data_outs

import libra_py.dynamics.qtag.qtag_basis as qtag_basis
import libra_py.dynamics.qtag.qtag_calc as qtag_calc
import libra_py.dynamics.qtag.qtag_checks as qtag_checks
import libra_py.dynamics.qtag.qtag_ham as qtag_ham
import libra_py.dynamics.qtag.qtag_init as qtag_init
import libra_py.dynamics.qtag.qtag_mom as qtag_mom
import libra_py.dynamics.qtag.qtag_prop as qtag_prop


In [7]:
class tmp:
    pass

#Holstein5 from libra_py/models/Hostein.py
def holstein_mimic(q, params, full_id):
    critical_params = ["E_n", "x_n", "k_n" ]
    default_params = { "V": [ [0.001, 0.001, 0.001, 0.001], [0.001, 0.001, 0.001, 0.001],
                              [0.001, 0.001, 0.001, 0.001], [0.001, 0.001, 0.001, 0.001] ],
                       "alpha": [ [0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00],
                                  [0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00] ],
                       "x_nm": [ [0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00],
                                 [0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00] ],
                     }
    comn.check_input(params, default_params, critical_params)

    E_n = params["E_n"]
    x_n = params["x_n"]
    k_n = params["k_n"]
    V = params["V"]
    alpha = params["alpha"]
    x_nm = params["x_nm"]
    
    n = len(E_n)

    Hdia = CMATRIX(n,n)
    Sdia = CMATRIX(n,n)
    d1ham_dia = CMATRIXList();  d1ham_dia.append( CMATRIX(n,n) )
    dc1_dia = CMATRIXList();  dc1_dia.append( CMATRIX(n,n) )

    
    Id = Cpp2Py(full_id)
    indx = Id[-1]
    
    #print(indx)

    x = q.col(indx).get(0)

    
    #indx = 0
    #x = q.col(indx).get(0)

    Sdia.identity()

    for i in range(n):
        Hdia.set(i,i,  (E_n[i] + 0.5*k_n[i]*(x - x_n[i])**2) * (1.0+0.0j) )

    for i in range(n):
        for j in range(n):
            if i!=j:
                Hdia.set(i,j,  V[i][j] * math.exp(-alpha[i][j] * (x-x_nm[i][j])**2 ) * (1.0+0.0j) )

    for k in [0]:
        #  d Hdia / dR_0
        for i in range(n):
            d1ham_dia[k].set(i,i, (k_n[i] * (x - x_n[i]))*(1.0+0.0j) )

    for k in [0]:
        for i in range(n):
            for j in range(n):
                if i!=j:
                    d1ham_dia[k].set(i,j,  -2.0*alpha[i][j] * (x-x_nm[i][j]) * V[i][j] * math.exp(-alpha[i][j] * (x-x_nm[i][j])**2 ) * (1.0+0.0j) )

    obj = tmp()
    obj.ham_dia = Hdia
    obj.ovlp_dia = Sdia
    obj.d1ham_dia = d1ham_dia
    obj.dc1_dia = dc1_dia

    return obj

In [8]:
def compute_model(q, model_params, full_id):

    model = model_params["model"]
    res = None
    
    if model==1:
        #res = Holstein.Holstein2(q, model_params, full_id)
        res = holstein_mimic(q, model_params, full_id)
    else:
        print("Unrecognized model! Check for correct model import statement.")

    return res

In [9]:
def run_qtag(dyn_params, _compute_model, _model_params, prefix):
    
#    Some checks for the user input...
#    comn.check_input(univ,{"ndof":1,"mass":[1836.0],"dt":0.1,"niter":1,"n_data_out":1,"n_snapshots":1},[])
#    comn.check_input(wf0,{},["q","p","a","s"])
#    comn.check_input(traj0,{"rho":1e-10},["placement","grid_dims","a0"])
#    comn.check_input(basis,{"qtype":"adpt","ptype":"adpt","atype":"adpt","stype":"frzn"},[])
#    comn.check_input(mss,{"decpl":0.9},["prop_method"])
#    comn.check_input(mom_params,{"adjust":"unmodified","beta":1e-1},[])
#    comn.check_input(model,{"calc_type":"LHA","coupling":"LHA"},["pot_type","rep"])
#    qtag_init.qtag_checks(univ,wf0,traj0,mss,model,model_params)

    #Define function objects to be used in various parts of the calculation...
    initialize, props, vapprox, mom_calc, propagate = qtag_checks.assign_fobj(dyn_params)

    #Rename variables locally for convenience...
    ndof = dyn_params['ndof']
    nstates = dyn_params["nstates"]
    active_state = dyn_params["active_state"]
    niter = dyn_params['niter']
    dt = dyn_params['dt']
    mass = dyn_params['mass']

    #Initialize the basis parameters {q,p,a,s} and create initial b-matrix...
    try:
        ntraj, qpas = initialize(ndof, nstates, dyn_params['traj0'], dyn_params['wf0'])
    except AttributeError:
        sys.exit("Error in traj0 dictionary: 'placement' keyword not recognized!")

    #Create initial projection vector b...
    bt = qtag_init.coeffs(dyn_params['wf0'],qpas, active_state)

    #Start simulation and walltime variables...
    walltime_start = time.time()
    t=0.0

    print(nstates, ndof, ntraj)
    
    ham = nHamiltonian(nstates, nstates, ndof)
    ham.add_new_children(nstates, nstates, ndof, ntraj)
    ham.init_all(2,1)
    model_params.update({"timestep":0})

    super_ovlp = CMATRIX(ntraj, ntraj)
    super_ham = CMATRIX(ntraj, ntraj)
    
    coeff = CMATRIX(nstates, ntraj)
    
        
    #ham.compute_diabatic(_compute_model, qpas[0], _model_params, 1)
    
    prms = dyn_control_params()
    prms.set_parameters(dyn_params);

    
    #sys.exit(0)
    
    for iter in range(niter):
        
        
        #Calculate system overlap on each state, placed into a super-matrix ovlp...
        #ovlp = qtag_calc.super_overlap(ndof,ntraj,nstates,qpas)

        #Calculate system Hamiltonian for all states and couplings, placed into a super-matrix hmat...
        #hmat = qtag_ham.super_hamiltonian(ndof,ntraj,nstates,mass,qpas,vapprox,compute_model,model_params)
        
        print(qpas[4], qpas[0].num_of_cols  )
        print(qpas[1].num_of_cols, qpas[2].num_of_cols, qpas[3].num_of_cols )
        
        #sys.exit(0)
        
        qtag_hamiltonian_and_overlap(qpas[0], qpas[1], qpas[2], qpas[3], coeff, 
                                     Py2Cpp_int(qpas[4]), iM, ham,
                                     _compute_model, _model_params, dyn_params,
                                     super_ovlp, super_ham)
        
        #data_outs.print_matrix(super_ovlp)
        
        #sys.exit(0)
        
        #Compute the new coefficient vector c_new...
        ct_new = qtag_ham.basis_diag(ntraj,dt, super_ham, super_ovlp, bt)
            
        etot = qtag_calc.energy(ct_new,hmat)
        pops = qtag_calc.norm(qpas[4],ct_new,ovlp,
                              [state for state in range(nstates)])

        print(etot)
        #Update the basis parameters according to the new wavefunction (ct_new)...
        qpasn,bt=propagate(dyn_params,qtag_params['mss'],mom_calc,
                           props,model_params,qpas,ct_new,pops,
                           dyn_params['mom_params']['beta'])

    print(pops)
    walltime_end = time.time()
    print("Total wall time: ",walltime_end-walltime_start)


In [ ]:
#Parameters for general simulation
dyn_params = {
    "ndof" : 1, "nstates" : 2, "active_state" : 0,
    "dt":0.01, "niter":50, "mass":[1.0],
    "nsnapshots":1, "noutput":10,
    "traj0":{"placement":"grid", "grid_dims" : [25], "a0":[6.0], "rho":1e-12},
    "basis":{"qtype" : "adpt", "ptype" : "adpt", "atype" : "adpt", "stype" : "frzn"},
    "wf0":{"q" : [-2.0], "p" : [0.0], "a" : [1.0], "s" : [0.0]},
    "mss":{"prop_method" : "sync", "decpl" : 0.3, "mirror" : "true"},
    "mom_params":{"adjust" : "lin_fit", "beta" : 1e-3},
    "v_approx":'BAT',
    "qtag_pot_approx_method":0
}

iM = MATRIX(1, 1);  iM.set(0,0, 1.0);

#QTAG-specific parameters, mostly function objects
#qtag_params = {
#    "traj0":{"placement":"grid", "grid_dims" : [25], "a0":[6.0], "rho":1e-12},
#    "basis":{"qtype" : "adpt", "ptype" : "adpt", "atype" : "adpt", "stype" : "frzn"},
#    "wf0":{"q" : [-2.0], "p" : [0.0], "a" : [1.0], "s" : [0.0]},
#    "mss":{"prop_method" : "sync", "decpl" : 0.3, "mirror" : "true"},
#    "mom_params":{"adjust" : "lin_fit", "beta" : 1e-3},
#    "v_approx":'BAT',
#    "qtag_pot_approx_method":0
#        
#}

#Potential parameters for Holstein5
model_params = {
    "model":1,
    "E_n":[0.0,15.8114],
    "x_n":[0.0,1.0],
    "k_n":[10.0,10.0],
    "V":[[0.0,1.0],[1.0,0.0]],
    "alpha":[[0.0,1.5811],[1.5811,0.0]],
    "x_nm":[[0.0,2.0],[2.0,0.0]]
}

run_qtag(dyn_params, compute_model, model_params, "qtag")